## 패키지 호출

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook
import re
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## DataFrame 전처리 함수

In [3]:
# 태그 전처리(지역정보제거)
def location(cat):
  for i in range(len(cat)):
    try:
      cat['keyword'][i] = str(cat['keyword'][i]).replace('대전','')
      cat['keyword'][i] = str(cat['keyword'][i]).replace('서울','')
      cat['keyword'][i] = str(cat['keyword'][i]).replace('대구','')
      cat['keyword'][i] = str(cat['keyword'][i]).replace('울산','')
      cat['keyword'][i] = str(cat['keyword'][i]).replace('부산','')
      cat['keyword'][i] = str(cat['keyword'][i]).replace('경기도','')
      cat['keyword'][i] = str(cat['keyword'][i]).replace('수지','')
      cat['keyword'][i] = str(cat['keyword'][i]).replace('NaN','')
    except: pass
  return(cat)
# 데이터 merge
def merge_df(df):
    df.fillna(0,inplace=True)
    df.replace(0,'',inplace=True)
    df['merge'] = df['title'] + ' '+df['cat1'] #+ df['cat2'] + df['cat3']
    df = df[['merge','keyword']]
    df.rename(columns={'merge':'feature', 'keyword' : 'label'}, inplace=True)
    return df

# 태그만 모아두는 리스트 생성 및 10번이상 노출된 태그 list 추출
def tag(cat):
  test = list(cat['label'])
  label_unique=''
  for i in range(len(test)):
    try : 
      label_unique += test[i]
    except : 
      pass
  label_unique = label_unique.split(',')
  label_unique = [v for v in label_unique if v]
  
  df = pd.DataFrame({'label_unique':label_unique, 'count':0}).groupby(['label_unique']).count().reset_index()
  df = df[df['count']>10].sort_values(by='count',ascending=False).reset_index()[['label_unique','count']]
  top = list(df['label_unique'])

  return(top)

# 태그 분배
def split_label(cat):
  for i in range(1,6):
    cat[f'label{i}'] = cat.label.str.split(',').str[i]
  cat = cat.fillna(0)
  return(cat)  

## 정규표현식 전처리(카테고리별 따로 지정)

In [4]:
def proccessing(df):
  for i in range(len(df)):
    ## 노트북 한정
    if '인치' in df.feature[i]:
      df.feature[i] = re.sub('\W+',' ', df.feature[i].lower())
    else : 
      df.feature[i] = re.sub('\W+',' ', df.feature[i].lower())
      df.feature[i] = re.sub('[0-9]',' ', df.feature[i].lower())
  return(df)

## tfidf 처리

In [5]:
def tfidf(proccessed_cat):
  from sklearn.feature_extraction.text import CountVectorizer
  vect = CountVectorizer()
  countvect = vect.fit_transform(proccessed_cat)

  countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))

  from sklearn.feature_extraction.text import TfidfVectorizer
  vect = TfidfVectorizer(max_features = 10000)
  tfvect = vect.fit(proccessed_cat)

  tfidv_df = pd.DataFrame(tfvect.transform(proccessed_cat).toarray(), columns = sorted(vect.vocabulary_))
  return(tfidv_df)

def make_matrix(tag_,cat):
  total_lab = pd.DataFrame()
  for label in tqdm(tag_):
    temp = pd.DataFrame()
    for i in range(len(cat)):
        if cat['label1'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label2'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label3'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label4'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
        elif cat['label5'][i] == label:
            temp = pd.concat([temp, cat.iloc[[i]]])
    try:
      temp_tfidf = tfidf(temp['feature'])
      temp_tfidf['label'] = label
      temp_tfidf = temp_tfidf.groupby('label').mean().reset_index()
      total_lab = pd.concat([total_lab,temp_tfidf])
    except: pass
  total_lab = total_lab.fillna(0)

  return(total_lab)

## 입력 및 태그 추출

In [6]:
# title = [input('제목을 입력해주세요 : ')]
def find_tag(title, cat_matrix):
  # test = proccessed_input(list(title))
  test = title[0].split(' ')
  for i, j in enumerate(test):
    i = re.sub('\W+',' ', j)

  test_input = []
  for i in test:
    if i in cat_matrix.columns:
      test_input.append(i)
  test_input.append('label')

  # 해당 카테고리에 적용
  test_matrix = cat_matrix[test_input]
  test_matrix

  # 상위 top5 태그 추출
  test_matrix['target'] = 0
  for i in range(len(test_input)-1):
    test_matrix['target'] += test_matrix[test_input[i]] 
  test_matrix['target'] = test_matrix['target'] / (len(test_input)-1)
  test_matrix = test_matrix[['label','target']].sort_values(by='target', ascending=False)[:5]
  return(test_matrix)

## 작업 자료 불러오기

In [7]:
# 작업 목록 df로 불러오기
query_list = ['갤럭시','구두','노트북','데스크탑','마우스','모자','바지','상의','셔츠','스니커즈','시계','아이폰','안경','에어컨',
              '에어팟','오토바이','원피스','전자제품','점퍼','중고차','쥬얼리','지갑','치마','카메라','코트','키보드','티비']

total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/1조/태그크롤링데이터/{query_list[i]}.csv'))

In [49]:
lab = total_df[2]
# 수기 전처리
lab = lab[(lab['cat1']=='디지털/가전')]
lab = lab[(lab['cat2']=='PC/노트북')]
lab = lab[(lab['cat3']=='노트북/넷북') | (lab['cat3']=='기타(PC/노트북)') | (lab['cat3']=='0')]
lab = location(lab)
lab.dropna(inplace=True)
lab.index=range(len(lab))
lab

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A va

,product_id,title,keyword,cat1,cat2,cat3,view,image
0,171860206,[전문]A급 8세대 1050 램12기가 옵테인500 한성 게이밍노트북,"노트북,게이밍,노트북,msi,보스몬스터",디지털/가전,PC/노트북,노트북/넷북,677,https://media.bunjang.co.kr/product/171860206_...
1,171044889,[전문]A급 i7 램16기가 ssd듀얼보스몬스터 게이밍노트북,"노트북,msi,한성노트북,게이밍,노트북",디지털/가전,PC/노트북,노트북/넷북,719,https://media.bunjang.co.kr/product/171044889_...
2,152053798,[전문]17인치 S급 풀박 i7 램16기가 ssd512 게이밍노트북,"노트북,msi,게이밍,노트북,1660ti",디지털/가전,PC/노트북,노트북/넷북,2892,https://media.bunjang.co.kr/product/152053798_...
3,152043062,[전문]A급 배터리신품 초슬림 i3 램8기가 ssd 240 hp노트북,"노트북,15인치,hp 노트북,인강용",디지털/가전,PC/노트북,노트북/넷북,1785,https://media.bunjang.co.kr/product/152043062_...
4,174538693,LG 노트북 13.3인치 팬티엄,"엘지,엘지그램,노트북,엘지노트북,삼성노트북",디지털/가전,PC/노트북,노트북/넷북,27,https://media.bunjang.co.kr/product/174538693_...
...,...,...,...,...,...,...,...,...
4246,170928907,삼성 노트북 실사용 5회미만,nan,디지털/가전,PC/노트북,노트북/넷북,105,https://media.bunjang.co.kr/product/170928907_...
4247,172834182,에이서 노트북,"에이서노트북,에이서,acer",디지털/가전,PC/노트북,노트북/넷북,49,https://media.bunjang.co.kr/product/172834182_...
4248,173905950,HP 15.6인치 게이밍 터치스크린 노트북 판매합니다,"터치스크린노트북,hp게이밍노트북,hp터치스마트,노트북,게이밍노트북",디지털/가전,PC/노트북,노트북/넷북,62,https://media.bunjang.co.kr/product/173905950_...
4249,172847961,레노버 노트북,nan,디지털/가전,PC/노트북,노트북/넷북,218,https://media.bunjang.co.kr/product/172847961_...


In [51]:
lab = merge_df(lab)
lab

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,feature,label
0,[전문]A급 8세대 1050 램12기가 옵테인500 한성 게이밍노트북 디지털/가전,"노트북,게이밍,노트북,msi,보스몬스터"
1,[전문]A급 i7 램16기가 ssd듀얼보스몬스터 게이밍노트북 디지털/가전,"노트북,msi,한성노트북,게이밍,노트북"
2,[전문]17인치 S급 풀박 i7 램16기가 ssd512 게이밍노트북 디지털/가전,"노트북,msi,게이밍,노트북,1660ti"
3,[전문]A급 배터리신품 초슬림 i3 램8기가 ssd 240 hp노트북 디지털/가전,"노트북,15인치,hp 노트북,인강용"
4,LG 노트북 13.3인치 팬티엄 디지털/가전,"엘지,엘지그램,노트북,엘지노트북,삼성노트북"
...,...,...
4246,삼성 노트북 실사용 5회미만 디지털/가전,nan
4247,에이서 노트북 디지털/가전,"에이서노트북,에이서,acer"
4248,HP 15.6인치 게이밍 터치스크린 노트북 판매합니다 디지털/가전,"터치스크린노트북,hp게이밍노트북,hp터치스마트,노트북,게이밍노트북"
4249,레노버 노트북 디지털/가전,nan


## 한번에 구현(1차 카테고리 수기 전처리 이후)
태그 전처리 및 전체 feature 전처리

In [54]:
lab = proccessing(lab)
lab

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,feature,label
0,전문 a급 세대 램 기가 옵테인 한성 게이밍노트북 디지털 가전,"노트북,게이밍,노트북,msi,보스몬스터"
1,전문 a급 i 램 기가 ssd듀얼보스몬스터 게이밍노트북 디지털 가전,"노트북,msi,한성노트북,게이밍,노트북"
2,전문 17인치 s급 풀박 i7 램16기가 ssd512 게이밍노트북 디지털 가전,"노트북,msi,게이밍,노트북,1660ti"
3,전문 a급 배터리신품 초슬림 i 램 기가 ssd hp노트북 디지털 가전,"노트북,15인치,hp 노트북,인강용"
4,lg 노트북 13 3인치 팬티엄 디지털 가전,"엘지,엘지그램,노트북,엘지노트북,삼성노트북"
...,...,...
4246,삼성 노트북 실사용 회미만 디지털 가전,nan
4247,에이서 노트북 디지털 가전,"에이서노트북,에이서,acer"
4248,hp 15 6인치 게이밍 터치스크린 노트북 판매합니다 디지털 가전,"터치스크린노트북,hp게이밍노트북,hp터치스마트,노트북,게이밍노트북"
4249,레노버 노트북 디지털 가전,nan


In [ ]:
## 수기로 feature 걸러낼 list 생성
cat = lab.copy()
cat = cat.cumsum()
cat = cat.feature.iloc[-1].split(" ")
temp =[]
for i in cat:
  if i not in temp:
    temp.append(i)
temp.sort()
temp

In [64]:
test =[]
for i in temp:
  if (len(i) > 3) and (i[0] != '0'):
    test.append(i)
test

['100s',
 '1050g1',
 '1060그램',
 '1080',
 '10th',
 '10세대',
 '11uek',
 '11세대',
 '11인치',
 '120g',
 '120gb',
 '128g',
 '128gb',
 '128기가',
 '12인치',
 '13u360',
 '13u580',
 '13z940',
 '13zd940',
 '13인치',
 '13인치노트북',
 '13인치슬림형',
 '13인치형',
 '14ibr',
 '14t360그램',
 '14u360',
 '14u53',
 '14z90n',
 '14z90p',
 '14z950',
 '14z960',
 '14z990',
 '14zd970',
 '14zd990',
 '14인같은',
 '14인치',
 '14인치노트북팝니다',
 '15iap',
 '15n530',
 '15u340',
 '15u480',
 '15u530',
 '15u560',
 '15z90n',
 '15z950',
 '15z95n',
 '15z980',
 '15z990',
 '15zb995',
 '15zd90n',
 '15zd950',
 '15zd970',
 '15인치',
 '15인치팝니다',
 '1660ti',
 '16gb',
 '16z90p',
 '16기가',
 '16년도',
 '16인치',
 '16인치화이트컬러',
 '17iml',
 '17z90p',
 '17zd95n',
 '17년도',
 '17인치',
 '17인치120hz고성능',
 '17인치17ud790',
 '19년형',
 '1tb고사향',
 '1회잠깐사용',
 '2008',
 '2010년',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2015년식',
 '2016',
 '2017',
 '2018',
 '2018년구매',
 '2018년도',
 '2018년식',
 '2018년형',
 '2019',
 '2019년형',
 '2020',
 '2020년',
 '2020년형',
 '2021',
 '2021버전',
 '202b',
 '20년제조',


In [65]:
len(test)

2158

In [47]:
# 데이터 처리를 위한 df 설정
lab = proccessing(lab)
lab = split_label(lab)
tag_list = tag(lab)
lab_matrix = make_matrix(tag_list,lab)
lab_matrix

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  0%|          | 0/103 [00:00<?, ?it/s]

,label,10th,10세대,11,11세대,11인치,120g,128gb,128기가,12인치,13,13z940,13zd940,13인치,13인치슬림형,14,14z90p,14인치,15,15u530,15u560,15z950,15z95n,15zd90n,15인치,16gb,16기가,16인치,17,17년,17년도,17인치,1tb,1등인기상품,1인치,1테라,2008,2011,2012,2012맥북프로,...,태클라스트,1kg,ux331u,고사양노트북,고사양노트북13인치,올려욥,dx,usb부품,x4j6y,삼성아티브북m,매입하고,미개봉노트북매입,스마트샵,스마트폰,있습니다,독서실,인스트론,헐값처분,골드라벨,uc,xkr,eg,ux534ftc304,먼지,않는다면,제거를,하지,dell인스피론,s펜,모드,테플릿,14z90n,eb3wk,사은품다드림,아티브북3,그래픽카드,gb노트북,p54g노트북,z400,고해상도14인치
0,노트북,0.000392,0.004782,0.001834,0.002004,0.000615,0.001638,0.000519,0.000421,0.000715,0.002808,0.000393,0.000474,0.018701,0.000342,0.000666,0.00043,0.010846,0.013178,0.000438,0.000416,0.000364,0.000441,0.000408,0.019211,0.001858,0.00043,0.005860,0.001257,0.000306,0.00059,0.008704,0.001061,0.000421,0.000321,0.000940,0.000314,0.000487,0.000362,0.000674,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
0,삼성노트북,0.000000,0.006250,0.000000,0.000000,0.001629,0.001595,0.000000,0.000000,0.002923,0.005553,0.000000,0.000000,0.012850,0.000000,0.000000,0.00000,0.016443,0.022266,0.000000,0.000000,0.000000,0.000000,0.000000,0.019940,0.000000,0.00000,0.003231,0.000000,0.000000,0.00000,0.001643,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
0,중고노트북,0.000000,0.003015,0.002875,0.000000,0.002875,0.003522,0.003173,0.000000,0.004727,0.003367,0.001809,0.000000,0.013697,0.001680,0.000000,0.00000,0.037411,0.033969,0.000000,0.000000,0.000000,0.000000,0.001976,0.020240,0.000000,0.00000,0.010388,0.001430,0.000000,0.00000,0.003078,0.000000,0.002041,0.000000,0.001625,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
0,게이밍노트북,0.000000,0.004435,0.001483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003445,0.000000,0.000000,0.003007,0.000000,0.000000,0.00000,0.005266,0.011762,0.000000,0.000000,0.000000,0.000000,0.000000,0.008348,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.013998,0.000000,0.000000,0.000000,0.003582,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
0,삼성,0.000000,0.000000,0.000000,0.000000,0.005184,0.002298,0.000000,0.000000,0.000000,0.005825,0.000000,0.000000,0.009297,0.000000,0.002725,0.00000,0.019782,0.016927,0.000000,0.000000,0.000000,0.000000,0.000000,0.019514,0.000000,0.00000,0.002116,0.002298,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,갤럭시,0.000000,0.000000,0.025363,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028549,0.000000,0.000000,0.028746,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025363,0.025363,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## 실제 데이터 입력 후 구현

In [25]:
title = [input('제목을 입력해주세요 : ')]
tag = find_tag(title, cat_matrix = lab_matrix)
tag

제목을 입력해주세요 : 2020 맥북프로 13인치 중고가 10세대


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,label,target
0,13인치,0.044764
0,맥북프로,0.043663
0,맥북에어,0.035593
0,17인치,0.034914
0,13인치노트북,0.033350
